# HW2 (프로그래밍언어론 2022년 1학기)

- 이름: 김승혜
- 학번: 20190595

이름과 학번을 위에 작성하시오.

----

In [1]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables #-}

In [2]:
data RE = Null
        | Epsilon
        | Symbol Char
        | Union RE RE
        | Concat RE RE
        | Star RE
        deriving (Eq,Ord)
-- https://stackoverflow.com/questions/27471937/showsprec-and-operator-precedences
instance Show RE where
  showsPrec _ Null    = showString "\\varnothing{}"
  showsPrec _ Epsilon = showString "\\varepsilon{}"
  showsPrec p (Symbol c) = showString [c]
  showsPrec p (Union  r1 r2) = showParen (p > 6) $
      showsPrec 6 r1 . showString " | " . showsPrec 6 r2
  showsPrec p (Concat r1 r2) = showParen (p > 7) $
      showsPrec 7 r1 . showsPrec 7 r2
  showsPrec p (Star r) = showParen (p > 10) $
      showsPrec 11 r . showString "^{*}"

import IHaskell.Display

instance IHaskellDisplay RE where
  display r = display [latex $ "$"++show r++"$"]
  
import Data.List (intercalate)
{-# LANGUAGE FlexibleInstances #-}
instance {-# OVERLAPPING #-} IHaskellDisplay [RE] where
    display rs = display . latex $ "$[\\;"++intercalate ",~" [show r | r <- rs]++"\\;]$"

In [3]:
:type latex
:type display

latex :: String -> DisplayData

display :: forall a. IHaskellDisplay a => a -> IO Display

In [4]:
Null
Epsilon
Symbol 'a'

In [5]:
Union (Symbol 'a') Epsilon
Union (Symbol 'a') (Union Epsilon Epsilon)
Union (Union Epsilon Epsilon) (Symbol 'a')
Union (Symbol 'a') (Concat Epsilon Epsilon)
Union (Concat Epsilon Epsilon) (Symbol 'a')

In [6]:
Concat (Symbol 'a') (Concat Epsilon Epsilon)
Concat (Concat Epsilon Epsilon) (Symbol 'a')
Concat (Symbol 'a') (Union Epsilon Epsilon)
Concat (Union Epsilon Epsilon) (Symbol 'a')

In [7]:
Star $ Symbol 'a'
Star $ Union (Symbol 'a') Epsilon
Star $ Concat (Symbol 'a') Epsilon
Star $ Star $ Symbol 'a'

----
## 정규식 간소화 비결정적 의미구조 계산기 (150점)
$$    \frac{~ ~}{~ \varnothing\,\pmb\mid\;r_2 \longmapsto r_2 ~}
\quad \frac{~ ~}{~ r_1\pmb\mid\,\varnothing \longmapsto r_1 ~}
\quad \frac{~ ~}{~ \varepsilon\,r_2 \longmapsto r_2 ~}
\quad \frac{~ ~}{~ r_1\,\varepsilon \longmapsto r_1 ~}
\quad \frac{~ r \longmapsto r' ~}{~ r^{*} \longmapsto r'^{*} ~}
$$
$$    \frac{~r_1 \longmapsto~ r_1' ~}{~ r_1\pmb\mid\,r_2 \longmapsto r_1'\pmb\mid\,r_2 ~}
\quad \frac{~r_2 \longmapsto~ r_2' ~}{~ r_1\pmb\mid\,r_2 \longmapsto r_1\pmb\mid\,r_2' ~}
\quad \frac{~r_1 \longmapsto~ r_1' ~}{~ r_1r_2 \longmapsto r_1'r_2 ~}
\quad \frac{~r_2 \longmapsto~ r_2' ~}{~ r_1r_2 \longmapsto r_1r_2' ~}
$$
위 작은걸음 규칙에 따라 정규식을 간소화하는 계산기 함수 `step`을 작성하라.

그리고 `step`을 확장한 `step'` 함수 및 수업시간에 다룬 `iterate` 함수 등을 활용해 모든 규칙을 최소한 한 번씩 사용하며
여러 단계에 걸쳐 간소화되는 정규식 예제를 하나 작성해 더 이상 간소회되지 않을 때까지 여러 단계에 걸쳐 정규식을 간소화하는 과정을
`mapM_`과 `print`를 활용해 출력하라.

In [8]:
step :: RE -> [RE]
step Null = []
step Epsilon = []
step (Symbol c) = []
step (Union Null r) = [r] ++ [Union Null r' | r' <- step r]
step (Union r Null) = [r] ++ [Union r' Null | r' <- step r]
step (Concat Epsilon r) = [r] ++ [Concat Epsilon r' | r' <- step r]
step (Concat r Epsilon) = [r] ++ [Concat r' Epsilon | r' <- step r]
step (Star r) = [Star r' | r' <- step r]
step (Union r1 r2) = [Union r1' r2 | r1' <- step r1] ++ [Union r1 r2' | r2' <- step r2]
step (Concat r1 r2) = [Concat r1' r2 | r1' <- step r1] ++ [Concat r1 r2' | r2' <- step r2]

In [9]:
import Data.List(nub)
hat f = \rs -> nub (concat [f r | r <- rs])

step' :: [RE] -> [RE]
step' = hat step

In [10]:
r0 = Concat (Union Null Epsilon `Concat` Symbol 'a')
            (Symbol 'b' `Concat` Union Epsilon Null)
    `Union`
    Star (Symbol 'c' `Concat` Epsilon)

In [11]:
rss = iterate step' [r0]

In [12]:
take 3 rss

In [13]:
takeWhile (not . null) rss

----

## 교재 그림 4.2의 의미함수를 하스켈로 옮겨 작성하라 (150점)
작성한 함수를 적절히 테스트한 내용도 포함해야 한다.
정규식의 6요소를 각 요소에 대해, 항상 `False`인 `Null`을 제외하고는
`True`인 경우와 `False`인 경우의 테스트를 해보아야 하므로, 최소 11개의 테스트는 있어야 할 것이다.
충분한 테스트를 제시하지 않으면 감점 대상이 된다. (테스트가 부족할 때마다 10점씩 감점)

함수가 정확히 구현되지 않으면 0점이며 정확하게 구현되지 않은 함수에 대한 테스트는 작성하더라도 0점이다.
왜나하면 부정확한 함수 작성에 대해 테스트를 통해 잘못된 점을 찾아냈어야 하는데
그렇지 못했다는 것은 테스트의 역할을 제대로 하지 못한 것이기 때문이다.
그러니까 정확하지 않은 구현에 대한 부분점수는 없다.

In [14]:
splitAt 2 "abcd"

("ab","cd")

In [15]:
splits s = [splitAt k s | k <- [0..length s]]

In [16]:
:type splits

splits :: forall a. [a] -> [([a], [a])]

In [17]:
splits "abcd"

[("","abcd"),("a","bcd"),("ab","cd"),("abc","d"),("abcd","")]

In [18]:
or [False,False,False,False]
or [False,False,True,False]

False

True

In [19]:
match :: RE -> String -> Bool
match Null _ = False
match (Epsilon) s = s == ""
match (Symbol a) s = s == [a]
match (Union r1 r2) s = match r1 s || match r2 s
match (Concat r1 r2) s = or [match r1 s1 && match r2 s2 | (s1, s2) <- splits s]
match (Star r) s = match (Union Epsilon (Concat r (Star r))) s

In [20]:
-- 테스트 작성
match Null ""
match Epsilon ""
match Epsilon "ab"
match (Symbol 'a') "a"
match (Symbol 'a') "ab"

False

True

False

True

False

In [21]:
match (Union Epsilon (Symbol 'a')) "a"
match (Union Epsilon (Symbol 'a')) ""
match (Union Epsilon (Symbol 'a')) "ab"

True

True

False

In [22]:
match (Concat (Symbol 'a') (Symbol 'b')) "ab"
match (Concat (Symbol 'a') (Symbol 'b')) "a"

True

False

In [23]:
match (Star (Union(Symbol 'a')(Symbol 'b'))) ""
match (Star (Union(Symbol 'a')(Symbol 'b'))) "a"
match (Star (Union(Symbol 'a')(Symbol 'b'))) "b"
match (Star (Union(Symbol 'a')(Symbol 'b'))) "c"
match (Star (Union(Symbol 'a')(Symbol 'b'))) "aa"
match (Star (Union(Symbol 'a')(Symbol 'b'))) "ab"
match (Star (Union(Symbol 'a')(Symbol 'b'))) "ba"
match (Star (Union(Symbol 'a')(Symbol 'b'))) "bb"
match (Star (Union(Symbol 'a')(Symbol 'b'))) "ac"

True

True

True

False

True

True

True

True

False

----
## 다음 성질을 만족하는 `revRE :: RE -> RE`를 작성하라 (150점)
$$[\![\textsf{revRE}(r)]\!](x) = [\![r]\!](x^R)$$
함수를 작성한 다음 의도한 대로 동작하는지 테스트해 보라.

In [24]:
revRE :: RE -> RE
revRE Null = Null
revRE Epsilon = Epsilon
revRE (Symbol a) = (Symbol a)
revRE (Union r1 r2) = Union (revRE r2) (revRE r1)
revRE (Concat r1 r2) = Concat (revRE r2) (revRE r1)
revRE (Star r) = (Star (revRE r))

In [25]:
ra = Symbol 'a'
rb = Symbol 'b'

r3 = Concat (Concat ra Epsilon) (Star (Union Null rb))

r3
revRE r3

In [26]:
match r3 ""
match r3 "a"
match r3 "ab"
match r3 "abb"
match r3 "aba"

False

True

True

True

False

In [27]:
r3R = revRE r3
match r3R $ reverse ""
match r3R $ reverse "a"
match r3R $ reverse "ab"
match r3R $ reverse "abb"
match r3R $ reverse "aba"

False

True

True

True

False